Zusammengefasst:

- Der Benutzer gibt seinen Namen und sein Geburtsdatum ein und wählt ein Bild aus.
- Das Programm berechnet das Sternzeichen basierend auf dem Geburtsdatum.
- Das Bild wird skaliert und zugeschnitten, das Sternzeichenbild hinzugefügt und der - Text mit Namen und Geburtsdatum über das Bild gelegt.
- Das kombinierte Bild wird angezeigt und kann gespeichert werden.

In [5]:
# Imports

import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from PIL import Image, ImageTk, ImageDraw, ImageFont
import numpy as np
from datetime import datetime


# Funktion zur Berechnung des Sternzeichens

# Diese Funktion berechnet das Sternzeichen basierend auf 
# dem Tag und Monat des Geburtsdatums.

def get_zodiac_sign(day, month):
    if (month == 12 and day >= 22) or (month == 1 and day <= 19):
        return "Steinbock"
    elif (month == 1 and day >= 20) or (month == 2 and day <= 18):
        return "Wassermann"
    elif (month == 2 and day >= 19) or (month == 3 and day <= 20):
        return "Fisch"
    elif (month == 3 and day >= 21) or (month == 4 and day <= 19):
        return "Widder"
    elif (month == 4 and day >= 20) or (month == 5 and day <= 20):
        return "Stier"
    elif (month == 5 and day >= 21) or (month == 6 and day <= 20):
        return "Zwilling"
    elif (month == 6 and day >= 21) or (month == 7 and day <= 22):
        return "Krebs"
    elif (month == 7 and day >= 23) or (month == 8 and day <= 22):
        return "Loewe"
    elif (month == 8 and day >= 23) or (month == 9 and day <= 22):
        return "Jungfrau"
    elif (month == 9 and day >= 23) or (month == 10 and day <= 22):
        return "Waage"
    elif (month == 10 and day >= 23) or (month == 11 and day <= 21):
        return "Skorpion"
    elif (month == 11 and day >= 22) or (month == 12 and day <= 21):
        return "Schütze"


# Bildauswahl-Funktion

# Diese Funktion öffnet einen Dateiauswahldialog, um ein Bild auszuwählen, 
# und zeigt es in der GUI an.

def select_image():
    print("Bild auswählen gestartet.")
    file_path = filedialog.askopenfilename()
    if file_path:
        try:
            print(f"Bildpfad: {file_path}")
            image = Image.open(file_path)
            image.thumbnail((800, 800), Image.LANCZOS)
            user_image = ImageTk.PhotoImage(image)
            image_label.config(image=user_image)
            image_label.image = user_image
            image_label.file_path = file_path
            print("Bild erfolgreich geladen und angepasst.")
        except Exception as e:
            print(f"Fehler beim Laden des Bildes: {e}")
            messagebox.showerror("Fehler", f"Bild konnte nicht geladen werden: {e}")


# Bildgröße anpassen und zuschneiden

# Diese Funktion skaliert und schneidet ein Bild auf eine Größe von 800x800 Pixeln zu.

def resize_and_crop_to_800x800(image_array):
    print("Bild wird auf 800x800 skaliert und zugeschnitten.")
    image = Image.fromarray(image_array)
    width, height = image.size
    aspect_ratio = width / height

    if aspect_ratio > 1:  # Landscape orientation
        new_height = 800
        new_width = int(aspect_ratio * new_height)
    else:  # Portrait orientation or square
        new_width = 800
        new_height = int(new_width / aspect_ratio)

    # Bild proportional skalieren
    image = image.resize((new_width, new_height), Image.LANCZOS)

    # Bild zuschneiden
    left = (new_width - 800) / 2
    top = (new_height - 800) / 2
    right = (new_width + 800) / 2
    bottom = (new_height + 800) / 2

    image = image.crop((left, top, right, bottom))
    return np.array(image)


# Ergebnisse anzeigen

# Diese Funktion sammelt die Benutzereingaben, 
# berechnet das Sternzeichen, 
# lädt und verarbeitet das Bild und das Sternzeichenbild, 
# fügt die Texte hinzu und speichert das kombinierte Bild.

def show_results():
    print("Ergebnisse anzeigen gestartet.")
    name = name_entry.get()
    birth_date = birth_date_entry.get()
    image_path = getattr(image_label, 'file_path', None)
   

    print(f"Name: {name}, Geburtsdatum: {birth_date}, Bildpfad: {image_path}")
    
    try:
        birth_date_obj = datetime.strptime(birth_date, "%d.%m.%Y")
        day, month = birth_date_obj.day, birth_date_obj.month
        zodiac_sign = get_zodiac_sign(day, month)
        print(f"Sternzeichen: {zodiac_sign}")
    except ValueError:
        print("Ungültiges Geburtsdatum.")
        messagebox.showerror("Fehler", "Ungültiges Geburtsdatum. Bitte im Format TT.MM.JJJJ eingeben.")
        return

    if not name or not birth_date or not image_path:
        print("Nicht alle Felder ausgefüllt oder kein Bild ausgewählt.")
        messagebox.showerror("Fehler", "Bitte füllen Sie alle Felder aus und wählen Sie ein Bild aus.")
        return
    
    # Erstellen des Ausgabe-Bildes
    try:
        user_image = Image.open(image_path)
        user_image = user_image.convert("RGBA")
        user_image_array = np.array(user_image)
        user_image_array = resize_and_crop_to_800x800(user_image_array)
        user_image = Image.fromarray(user_image_array)
        user_image_width, user_image_height = user_image.size
        print("Benutzerbild erfolgreich geladen und angepasst.")
    except Exception as e:
        print(f"Fehler beim Verarbeiten des Benutzerbildes: {e}")
        messagebox.showerror("Fehler", f"Bild konnte nicht verarbeitet werden: {e}")
        return

    # Laden und Einfügen des Sternzeichenbildes
    zodiac_image_path = f"zodiac_images/{zodiac_sign.lower()}.png"
    try:
    
        zodiac_image = Image.open(zodiac_image_path)
        zodiac_image.thumbnail((200, 200), Image.LANCZOS)
        zodiac_image = zodiac_image.convert("RGBA")


    # Alphakanal anpassen, um Transparenz hinzuzufügen
        alpha = zodiac_image.split()[3]
        alpha = Image.eval(alpha, lambda a: int(a * 0.5))
        zodiac_image.putalpha(alpha)


        zodiac_image_width, zodiac_image_height = zodiac_image.size

    except FileNotFoundError:
        messagebox.showerror("Fehler", f"Sternzeichenbild für {zodiac_sign} nicht gefunden.")
        return

        #zodiac_image = Image.open

        #print("Sternzeichenbild erfolgreich geladen und angepasst.")
    #except FileNotFoundError:
    #    print(f"Sternzeichenbild für {zodiac_sign} nicht gefunden.")
    #    messagebox.showerror("Fehler", f"Sternzeichenbild für {zodiac_sign} nicht gefunden.")
    #    return

    combined_image = Image.new('RGBA', (user_image_width, user_image_height))
    combined_image.paste(user_image, (0, 0))
    combined_image.paste(zodiac_image, (user_image_width - zodiac_image_width, 0), zodiac_image)

    # Hinzufügen von Name und Geburtsdatum
    draw = ImageDraw.Draw(combined_image)
    try:
        font = ImageFont.truetype("arial.ttf", 40)
        print("Schriftart Arial erfolgreich geladen.")
    except IOError:
        font = ImageFont.load_default()
        print("Standard-Schriftart geladen.")

    text = f"Name: {name}\nGeburtsdatum: {birth_date}\nDu bist: {zodiac_sign}"
    text_position = (10, user_image_height - 120)
    draw.text(text_position, text, font=font, fill="white")

    # Speichern des Ausgabe-Bildes
    print("Öffnen des Dialogfensters zum Speichern.")
    save_path = filedialog.asksaveasfilename(defaultextension=".png",
                                             filetypes=[("PNG files", "*.png"), ("All files", "*.*")])
    print(f"Speicherpfad: {save_path}")
    
    if save_path:
        try:
            print(f"Speichern des Bildes unter: {save_path}")
            combined_image.save(save_path)
            print(f"Bild erfolgreich gespeichert unter: {save_path}")
            messagebox.showinfo("Erfolg", f"Bild erfolgreich gespeichert unter: {save_path}")
        except Exception as e:
            print(f"Fehler beim Speichern des Bildes: {e}")
            messagebox.showerror("Fehler", f"Bild konnte nicht gespeichert werden: {e}")
    else:
        print("Speichervorgang abgebrochen.")

    # Anzeigen des Ausgabe-Bildes
    combined_image.show()


# GUI-Setup

# Dieser Teil des Codes erstellt die 
# GUI-Elemente (Labels, Eingabefelder, Buttons) 
# und startet die Hauptschleife der GUI.

root = tk.Tk()
root.title("Sternzeichen Bestimmung")

# Layout
name_label = ttk.Label(root, text="Name:")
name_label.grid(row=0, column=0, padx=10, pady=10)
name_entry = ttk.Entry(root)
name_entry.grid(row=0, column=1, padx=10, pady=10)

birth_date_label = ttk.Label(root, text="Geburtsdatum (TT.MM.JJJJ):")
birth_date_label.grid(row=1, column=0, padx=10, pady=10)
birth_date_entry = ttk.Entry(root)
birth_date_entry.grid(row=1, column=1, padx=10, pady=10)

# Bildanzeige
image_label = ttk.Label(root)
image_label.grid(row=2, column=0, columnspan=2, padx=10, pady=10)

# Bildauswahl-Button
select_image_button = ttk.Button(root, text="Bild auswählen", command=select_image)
select_image_button.grid(row=3, column=0, columnspan=2, padx=10, pady=10)

# Ergebnis-Button
show_results_button = ttk.Button(root, text="Ergebnisse anzeigen", command=show_results)
show_results_button.grid(row=4, column=0, columnspan=2, padx=10, pady=10)

# Hauptschleife starten
root.mainloop()

# Zusammengefasst:

# Der Benutzer gibt seinen Namen und sein Geburtsdatum ein und wählt ein Bild aus.
# Das Programm berechnet das Sternzeichen basierend auf dem Geburtsdatum.
# Das Bild wird skaliert und zugeschnitten, 
# das Sternzeichenbild hinzugefügt und der Text mit 
# Namen und Geburtsdatum über das Bild gelegt.
# Das kombinierte Bild wird angezeigt und kann gespeichert werden.

Bild auswählen gestartet.
Bildpfad: C:/Users/Admin/OneDrive - DataCraft GmbH/Bilder/Eigene Aufnahmen/1 ifw5E3XGtKl36U0xe0UnyA.jpg
Bild erfolgreich geladen und angepasst.
Ergebnisse anzeigen gestartet.
Name: dd, Geburtsdatum: 17.01.1968, Bildpfad: C:/Users/Admin/OneDrive - DataCraft GmbH/Bilder/Eigene Aufnahmen/1 ifw5E3XGtKl36U0xe0UnyA.jpg
Sternzeichen: Steinbock
Bild wird auf 800x800 skaliert und zugeschnitten.
Benutzerbild erfolgreich geladen und angepasst.
Schriftart Arial erfolgreich geladen.
Öffnen des Dialogfensters zum Speichern.
Speicherpfad: 
Speichervorgang abgebrochen.
